# ***Data Preprocessing & Cleaning*** 

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Load Dataset

In [2]:
df = pd.read_csv("../data/raw/online_retail.csv")

### Dataset first look

In [3]:
df

InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

                InvoiceDate  UnitPrice  CustomerID         Country  
0       2022-12-01 08:26:00       2.55     17850.0  United Kingdom  
1       2022-12-01 08:26:00       3.39     17850.0  United Kingdom  
2       2022-12-01 08:26:00       2.75     17850.0  United Kingdom  
3       2022-12-01 08:26:00       3.39     17850.0  United Kingdom  
4       2022-12-01 08:26:00       3.39     17850.0  United Kingdom  
...                     ...        ...         ...             ...  
541904  2023-12-09 12:50:00       0.85     12680.0          France  
541905  2023-12-09 12:50:00       2.10     12680.0          France  
541906  2023-12-09 12:50:00       4.15     12680.0          France  
541907  2023-12-09 12:50:00       4.15     12680.0          France  
541908  2023-12-09 12:50:00       4.95     12680.0          France  

[541909 rows x 8 columns]

### Dataset Information

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


### Duplicate value counts

In [10]:
df.duplicated().value_counts()

False    536641
True       5268
Name: count, dtype: int64

### Missing/Null values

In [11]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

### Describe dataset

In [12]:
df.describe()

Quantity      UnitPrice     CustomerID
count  541909.000000  541909.000000  406829.000000
mean        9.552250       4.611114   15287.690570
std       218.081158      96.759853    1713.600303
min    -80995.000000  -11062.060000   12346.000000
25%         1.000000       1.250000   13953.000000
50%         3.000000       2.080000   15152.000000
75%        10.000000       4.130000   16791.000000
max     80995.000000   38970.000000   18287.000000

## ***Data Cleaning For the Customer Segmentation Module***

### For the Customer Segmentation Module
#### This module relies on RFM (Recency, Frequency, and Monetary) analysis, which requires the InvoiceDate, CustomerID, Quantity, and UnitPrice columns to calculate customer behavior metrics. The following data cleaning steps are essential:


- Handling Missing Values: Remove all rows where CustomerID is missing. The dataset inspection showed that there are a significant number of transactions (over 135,000) that are not linked to a specific customer, so these should be removed to ensure accurate customer-level analysis.


- Removing Canceled Invoices: Exclude all transactions where the InvoiceNo starts with the letter 'C'. These represent returns or cancellations and should not be included in the purchase history.


- Filtering Invalid Quantities and Prices: Remove rows where Quantity or UnitPrice are less than or equal to zero. This ensures that only valid purchase data is used for calculating total spend and frequency.

- Data Type Conversion: Convert the InvoiceDate column from its current object type to a datetime format. This is a critical step for accurately calculating the Recency metric, which is the time since a customer's last purchase.

### Dropping rows where CustomerID is missing

In [14]:
df_cleaned = df.dropna(subset=['CustomerID'])

In [16]:
df_cleaned.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

### Dropping duplicate values

In [20]:
df_cleaned = df.drop_duplicates()

In [22]:
df_cleaned.duplicated().value_counts()

False    536641
Name: count, dtype: int64

### Exclude canceled invoices by filtering out rows where 'InvoiceNo' starts with 'C' because these represent returns or cancellations and should not be considered as a purchase.

In [23]:
df_cleaned = df[~df['InvoiceNo'].str.startswith('C', na=False)]

### Remove rows with negative or zero quantities and prices

In [24]:
df_cleaned = df_cleaned[(df_cleaned['Quantity'] > 0) & (df_cleaned['UnitPrice'] > 0)]

### Convert 'InvoiceDate' to datetime objects

In [25]:
df_cleaned['InvoiceDate'] = pd.to_datetime(df_cleaned['InvoiceDate'])

In [26]:
df_cleaned.shape

(530104, 8)